## n-gram 介绍

![](md-img/n-gram.jpg)

可以理解为，n-gram 就是按照步长为 1，截取某个序列的长度为 n 的子序列的集合

<br>

## Modified Precision 介绍

![](md-img/modifiedprecision1.jpg)

![](md-img/modifiedprecision2.jpg)

![](md-img/modifiedprecision3.jpg)

![](md-img/modifiedprecision4.jpg)

![](md-img/modifiedprecision5.jpg)

其中每次计数只考虑在机器翻译中出现的 n-gram

得到的 P1、P2、、、Pn 就是 Modified Precision

<br>

## 机器翻译评估指标 BLEU

BLEU —— Bilingual Evaluation Understudy

用于衡量机器翻译与人工翻译的相似度，BLEU 越大，表示模型效果越好

BLEU 的总体计算公式为：

$$
\text{BLEU} = \text{BP} \times \exp \left( \sum_{n=1}^{N} w_n \log p_n \right)
$$

其中：

- N：n-gram 的最大阶数（通常取 4，即 BLEU-4）  
- w_n：每个 n-gram 的权重，通常 w_n = 1 / N
- p_n：第 n 阶 n-gram 的精确率（Modified Precision）  

<br>

BP 是长度惩罚项

我们考虑若机器翻译只有一个词，而这个词在人工翻译中又恰好出现了，这会导致其精确度为 1

也就是说，机器翻译越短，计算的精确度越高，但往往不是期望的翻译结果

BP 的作用就是，让机器翻译结果短的 BLEU 值低一点

$$
\text{BP} =
\begin{cases}
1, & \text{if } c > r \\
e^{(1 - r/c)}, & \text{if } c \le r
\end{cases}
$$

其中 c 表示机器翻译长度，r 表示人工翻译长度

<br>

## BLEU 指标计算

In [1]:
import math
import collections

In [2]:
def bleu(pred_seq, label_seq, k):
    pred_tokens, label_tokens = pred_seq.split(' '), label_seq.split(' ')
    len_pred, len_label = len(pred_tokens), len(label_tokens)

    # 计算长度惩罚项
    bp = math.exp(min(0, 1 - len_label / len_pred))

    sm = 0
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[' '.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[' '.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[' '.join(pred_tokens[i: i + n])] -= 1
        p = num_matches / (len_pred - n + 1)
        sm += 1 / k * math.log(p)

    score = bp * math.exp(sm)
    return score